In [11]:
import subprocess
import os
import time
import json
import pandas as pd

working_dir = './results/reza_gpt2_layers/'
if not os.path.exists(working_dir):
    os.makedirs(working_dir)

if not os.path.exists('settings.json'):
    data = {"OPENAI_API_KEY": "YYY"}
    with open('settings.json', 'w') as f:
        json.dump(data, f, indent=4) 

with open('settings.json', 'r') as file:
    data = json.load(file)
    field_name = "OPENAI_API_KEY"
    OPENAI_API_KEY = data[field_name]

def run_pop8query(keywords, datasource, max_results, output_format, output_file):
    cmd = [
        "./assets/pop8query",
        "--keywords={}".format(keywords),
        "--{}".format(datasource),
        "--max={}".format(max_results),
        "--format={}".format(output_format),
        output_file
    ]

    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode != 0:
        print("Error occurred:", result.stderr)
    else:
        print("Command executed successfully!")
        print("Output:", result.stdout)

# Example usage:
# run_pop8query("machine learning", "semscholar", 20, "json", "output.json")


def get_papers(search_phrase, dataset="semscholar"):
    try:
        run_pop8query(search_phrase, dataset, 20, "json", "output.json")
        
        # Check if output.json was created and is not empty
        if not os.path.exists("output.json") or os.path.getsize("output.json") == 0:
            print(f"Error: Output file for '{search_phrase}' not created or is empty.")
            return pd.DataFrame()  # Return empty dataframe

        with open("output.json", "r", encoding="utf-8-sig") as file:
            data = json.load(file)

        if not data:
            print(f"No data found in the JSON file for '{search_phrase}'.")
            return pd.DataFrame()  # Return empty dataframe

        df = pd.DataFrame(data)

        return df

    except Exception as e:
        print(f"Error processing '{search_phrase}': {e}")
        return pd.DataFrame()  # Return empty dataframe in case of any other unexpected errors


In [2]:
import openai
import time
import tiktoken

class llmOperations:    
    total_prompt_tokens = 0
    total_cmpl_tokens = 0

    openai.api_key = OPENAI_API_KEY
    # openai.api_key = os.getenv('OPENAI_API_KEY').strip('"')
    # language_model = 'gpt-3.5-turbo-instruct-0914'
    # language_model = "babbage-002"

    def __init__(self, language_model="gpt-3.5-turbo-0613", price_inp=0.0015/1000, price_out=0.002/1000):
        self.language_model=language_model
        self.price_inp=price_inp
        self.price_out=price_out    
        self.tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")

    
    def get_llm_response(self, prompt, system_prompt = "You are a smart, very knowledgable, research assistant."):
        chat_data = [{'role': 'system', "content": system_prompt}, {'role': 'user', 'content': prompt}]
        # print(chat_data)
        
        response = openai.ChatCompletion.create(model=self.language_model, messages=chat_data)
        final_response = response['choices'][0]['message']['content']
        
        self.total_prompt_tokens += response['usage']['prompt_tokens']
        self.total_cmpl_tokens += response['usage']['completion_tokens']
        
        return final_response, response

    def get_current_cost(self):
        return self.total_prompt_tokens*self.price_inp + self.total_cmpl_tokens*self.price_out
        
    def get_estimated_cost(self, prompt, completion_estimate_len=100):
        # Assumes the system prompt is small, and prompt variable contains all text to be processed by LLM        
        return len(self.tokenizer.encode(prompt))*self.price_inp + completion_estimate_len*self.price_out

short_context_model = llmOperations()
long_context_model = llmOperations('gpt-3.5-turbo-16k', price_inp=0.003/1000, price_out=0.004/1000)
# response = openai.Completion.create(model=language_model, prompt=prompt, max_tokens=300, temperature=0)
# final_response = response['choices'][0]['text']



In [ ]:
idea_information = {}
papers_info = {}

In [3]:
# Get search phrases
idea_text = "One problem in self supervised learning without negative instances is collapse. To avoid collapse, I will use eigenvalues of the output embedding space and ensure they are all larger than 1.0 to make sure the space of embedding is used effectively. Still, making sure the variations of the same data point (generated by some augmentation) are mapped closer to one another. "
# idea_text = """The prevailing mental health predicament compels us to investigate resilience—the intrinsic capability to counteract stress and rejuvenate mental well-being. While resilience is recognized for its potent influence on mental health, aging, recovery from ailments, and possible deterrence of cognitive decline, there remains a significant void in understanding its transformation with age and its modulation by major life events. This research endeavors to delve into the neurocognitive mechanisms underpinning emotional resilience in older populations, both healthy and those with mood disorders. The central thrust is to discern how aging individuals process emotions, given its profound impact on their overall well-being. Harnessing a synergistic approach that melds brain imaging, cognitive assessments, physiological monitoring, and real-world data, the study seeks to: 1) Uncover the intricate neurological, cognitive, and physiological substrates bolstering emotional resilience in aging; 2) Contrast the neurological and physiological responses of older individuals with late-life depression against their healthy counterparts; and 3) Track and prognosticate the trajectory of mental well-being in the twilight years. Through these pursuits, this research aims to amplify our grasp on the intricate dance between aging, emotion, and cognition, ultimately steering the creation of innovative strategies to bolster mental health in senior years."""
# idea_text = """As people age, they may pay less attention to the social aspects of their environment2. Normative aging has a negative impact on certain aspects of social cognition and specifically in social perception. One notable difference between younger and older individuals is the age-related decline in perceiving and integrating social-emotional cues from faces. Most of the evidence, however, stems from studies in which the data is averaged across individuals, making conclusions about the two extreme ends of the age spectrum rather than a complete picture across the lifespan. In addition, processing social-emotional cues typically vary across individuals and across tests. Without current knowledge of how social perception is affected across the adult lifespan – from young to middle age to late adulthood – and without fully understanding individual differences in processing social-emotional cues (inter-individual differences) in different tests, it is difficult to draw conclusions as to why social perception change with age. The plan is to identify neurocognitive mechanisms underlying age-related differences in social perception function. I will achieve this by examining brain networks across the adult lifespan, from young adulthood to middle age and older adulthood to identify critical window in which this function starts to change. Due to the heterogeneity of aging population and individual variability of social-emotional processing, I will further investigate inter-individual differences in social perception functions. I will achieve this by having repeated measurements of social perception among few individuals, also known as deep sampling, to determine how individual variabilities could explain age-related differences observed in social perception. Furthermore, I will zoom in and investigate the role of subcortical brain structures, which have been largely overlooked in social cognitive functions due to technical challenges associated with accurate mapping of these areas. I will achieve this by measuring structural and functional properties of the amygdala and locus coeruleus to further understand their role in social perception functions."""

prompt = f'Here is a description of an idea: {idea_text}. \n Generate 5 search phrases to search in Google for related articles. Generate the search phrases in a json format, with fields of "search phrase X", where X is the number. Include nothing but this json format output in your response.'
final_response, response = short_context_model.get_llm_response(prompt)

parsed_data = json.loads(final_response)
search_phrases = []
for i in parsed_data.keys():
    search_phrases.append(parsed_data[i])
print(search_phrases)
# TODO: Save search keys

['self supervised learning without negative instances', 'embedding space collapse in self supervised learning', 'eigenvalues in self supervised learning', 'ensuring eigenvalues larger than 1.0 in embedding space', 'mapping variations of same data point in embedding space']


In [5]:
with open(working_dir + 'search_phrases.txt', 'w') as f:
    f.write('\n'.join(search_phrases))

In [6]:
short_context_model.get_current_cost()

0.0004075

In [7]:
prompt = "Summarize this research idea to a concise paragraph while make sure it does not loos any important message or question:\n"+idea_text
idea_text_summary, response = short_context_model.get_llm_response(prompt)

print(idea_text_summary)

This research idea addresses the issue of collapse in self-supervised learning, specifically when negative instances are not available. To prevent collapse, the proposal suggests using the eigenvalues of the output embedding space and ensuring they are all greater than 1.0. This aims to optimize the utilization of the embedding space. Additionally, the idea seeks to ensure that augmented variations of the same data point are mapped closer to each other.


You are an expert in the field of self-supervised learning and embedding spaces.


In [8]:
with open(working_dir + 'idea_summary.txt', 'w') as f:
    f.write('Idea: \n\n')
    f.write(idea_text)
    f.write('\n\n idea summary:\n\n')
    f.write(idea_text_summary)

In [12]:
import pandas as pd
# search_phrases = ["Neural network image embedding eigenvalues greater than 1.0",
# "Eigenvalue regularization in deep learning for common image embedding",
# "Spectral constraints in neural network embeddings for image sets",
# "Image distribution alignment with eigenvalue constraints in deep learning"]

# Find papers
combined_df = pd.DataFrame()
engines = ['gscholar', 'pubmed', 'semscholar']
# engines = ['pubmed']

for search_phrase in search_phrases:
    for engine in engines:
        print(f"Extracting papers with search phrase: {search_phrase} from {engine}")
        df = get_papers(search_phrase, )
        combined_df = pd.concat([combined_df, df])
        time.sleep(2)
    
    # df = get_papers(search_phrase, 'gscholar')
    # combined_df = pd.concat([combined_df, df])
    # time.sleep(2)
    
combined_df = combined_df.drop_duplicates(subset=['abstract'])
clean_df = combined_df.dropna(subset= ['abstract'])

Extracting papers with search phrase: self supervised learning without negative instances from gscholar
Command executed successfully!
Output: 
Extracting papers with search phrase: self supervised learning without negative instances from pubmed
Command executed successfully!
Output: 
Extracting papers with search phrase: self supervised learning without negative instances from semscholar
Command executed successfully!
Output: 
Extracting papers with search phrase: embedding space collapse in self supervised learning from gscholar
Command executed successfully!
Output: 
Extracting papers with search phrase: embedding space collapse in self supervised learning from pubmed
Command executed successfully!
Output: 
Extracting papers with search phrase: embedding space collapse in self supervised learning from semscholar
Command executed successfully!
Output: 
Extracting papers with search phrase: eigenvalues in self supervised learning from gscholar
Command executed successfully!
Output: 
E

In [14]:
clean_df.to_csv(working_dir+'papers_found.csv')

In [17]:
# Simulate cost for finding relevance scores
import tiktoken
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
token_out_estimate = 100
estimated_cost = 0
relevance_scores = []

# clean_df['abstract'].values.shape[0]
for i in range(clean_df.shape[0]):
    abstract = clean_df['abstract'].values[i]
    # print(abstract)
    # text = 'Here is an idea: ' + idea_text_summary + '\n \n' + "How relevant this idea is to the following abstract of a paper:\n" + abstract + """\n \nPick the relevance score from very low, low, medium, high, and very high. Output format as json, with field "relevance". Include nothing but this json format output in your response."""
    
    text = 'Here is an idea: ' + idea_text_summary + '\n \n' + "How relevant this idea is to the following abstract of a paper:\n" + abstract + """\n \nPick the relevance score from very low, low, medium, high, and very high. Output format as json, with field "relevance" and "reason". Include nothing but this json format output in your response."""
    parsed_response = """{
        "relevance": "very high",
        "reason": "Your idea aligns closely with the concepts discussed in the paper's abstract, which focuses on avoiding collapse and regularizing the covariance matrix of network outputs. Your idea of using eigenvalues to ensure consistent embeddings is directly related to the paper's content."
        }
    """   
    # parsed_response = """{
    #     "relevance": "very high",
    #     }
    # """    
    estimated_cost += short_context_model.get_estimated_cost(text, 100)

print("Estimated cost is: ", estimated_cost)

Estimated cost is:  0.067141


In [18]:
relevance_scores = []
real_cost = 0

for i in range(clean_df.shape[0]):
    abstract = clean_df['abstract'].values[i]
    
    prompt = 'Here is an idea: ' + idea_text_summary + '\n' + "How relevant this idea is to the following abstract of a paper:\n" + abstract + """\n \nPick the relevance score from very low, low, medium, high, and very high. Output format as json, with fields "relevance" and "reason", which would look like:\n
    {"relevance": "RELEVANCE", "reason": "THE REASON"}. Include nothing but this json format output in your response."""

    parsed_response, response = short_context_model.get_llm_response(prompt)

    try:
        parsed_data = json.loads(parsed_response)
    except:
        parsed_data = {"relevance": "unknown", "reason": parsed_response}
    columns_to_pass = ['authors', 'abstract', 'doi', 'cites', 'year', 'title']
    for c in columns_to_pass:
        try:
            parsed_data[c] = clean_df[c].values[i]
        except:
            parsed_data[c] = "NONE"
    
    relevance_scores.append(parsed_data)
    
    tokens_out = tokenizer.encode(parsed_response)
    real_cost = short_context_model.get_current_cost()
    print(i, len(tokens_out), ', cost so far:', real_cost)
    
print('your cost so far: ', short_context_model.get_current_cost())

0 74 , cost so far: 0.0015945
1 73 , cost so far: 0.002594
2 104 , cost so far: 0.003618
3 88 , cost so far: 0.0043175
4 114 , cost so far: 0.005132
5 109 , cost so far: 0.0061135
6 94 , cost so far: 0.0069735000000000005
7 81 , cost so far: 0.00789
8 90 , cost so far: 0.008721
9 76 , cost so far: 0.009743
10 91 , cost so far: 0.010554999999999998
11 95 , cost so far: 0.0113165
12 81 , cost so far: 0.0120665
13 102 , cost so far: 0.013058
14 96 , cost so far: 0.013847
15 55 , cost so far: 0.014752000000000001
16 66 , cost so far: 0.0155665
17 63 , cost so far: 0.016195
18 87 , cost so far: 0.016989999999999998
19 93 , cost so far: 0.0177805
20 71 , cost so far: 0.0185375
21 71 , cost so far: 0.019368
22 73 , cost so far: 0.0201935
23 73 , cost so far: 0.0208765
24 85 , cost so far: 0.021789
25 104 , cost so far: 0.022557999999999998
26 91 , cost so far: 0.023385
27 66 , cost so far: 0.024366000000000002
28 83 , cost so far: 0.025261000000000002
29 88 , cost so far: 0.0261045
30 82 , co

In [19]:
relevance_scores_df = pd.DataFrame(relevance_scores)

In [20]:
relevance_scores_df.to_csv(working_dir + '/first_level_analysis.csv')

In [80]:
min_cite = 100
min_year = 2020
filtered_df = relevance_scores_df[relevance_scores_df["relevance"].str.lower().isin(["high", "very high"])]
papers_df = filtered_df[(filtered_df['year']>min_year) | (filtered_df['cites']>min_cite)]
papers_df.shape

(19, 8)

In [65]:
prompt = "Here is a research proposal:\n"+idea_text+'\n If a professor is going to research this propsoal, what would the professor be expert at? Generate the answer in the format of "You are an expeert in the field of XXX"'
researcher_spec, response = short_context_model.get_llm_response(prompt)

print(researcher_spec)

You are an expert in the field of self-supervised learning and its challenges, specifically related to collapse and the utilization of eigenvalues in the output embedding space.


In [81]:
# concated_data = '\n next abstract:\n'.join(papers_df['abstract'].values.tolist())
large_mdl = 'gpt-4-0613'
# large_mdl = 'gpt-3.5-turbo-16k'
pricing_map = {'gpt-4-0613': [0.03/1000, 0.06/1000], 'gpt-3.5-turbo-16k': [0.003/1000, 0.004/1000], 'gpt-3.5-turbo-0613': [0.0015/1000, 0.002/1000]}
long_context_model = llmOperations(large_mdl, price_inp=pricing_map[large_mdl][0], price_out=pricing_map[large_mdl][1])

concated_data = [('Paper ID '+ str(i) + ': \n' + p + '\n\n') for i, p in enumerate(papers_df['abstract'].values.tolist())]
concated_data = ''.join(concated_data)
print('Estimated price for litrature review: ', long_context_model.get_estimated_cost(concated_data, 1500))


Estimated price for litrature review:  0.23063999999999998


In [82]:
with open(working_dir + 'used_papers_review.txt', 'w', encoding="utf-8-sig") as f:
    f.write(concated_data)
print(concated_data)

Paper ID 0: 
Recently, self-supervised learning methods have been proposed to learn a useful representation for visual detection of anomalous, unhealthy crops while a neural network classifies augmented images of normal instances, which are relatively easy-to-obtain. Their pipelines are largely designed within the one-class classification paradigm, in which training samples are all of normal (negative) class, considering the severe scarcity of anomalous (positive) observations in realistic scenarios. In this paper, we study whether this “homogeneity” of training set is necessary to boost up the performance of learned detector because otherwise “unlabeled” data newly gathered from the field could simply be utilized during training without the need of expensive human annotation. To be specific, we first explore the scenarios treating every unlabeled instance as a normal one as the proportion of anomalous samples in the unlabeled set varies. Also, we introduce an iterative training proced

In [83]:
# Write the litrature review
prompt = researcher_spec+'\n \n Here is an idea: ' + idea_text_summary + '\n' + "and here are some paper abstracts that are relevant to this idea:\n\n" + concated_data + """\n\n END OF PAPER ABSTRACT PROVIDED.\n \nWrite a litrature review section, which I will be using for my paper in the background section, using these papers about the idea. Use as much as these papers as you can. Ensure the review is engaging and compares the ideas, rather than a flat list of papers. Also, the review makes reference back to my idea where relevant. Use Paper IDs for referencing, for example [Paper ID 3]. Also, at the very end, add one short and condensed paragraph and discuss how my idea is going to advance the field further and what gaps it will be filling."""
parsed_response, response = long_context_model.get_llm_response(prompt)

print('your cost so far: ', long_context_model.get_current_cost())

your cost so far:  0.19199999999999998


In [84]:
print(prompt)

You are an expert in the field of self-supervised learning and its challenges, specifically related to collapse and the utilization of eigenvalues in the output embedding space.
 
 Here is an idea: This research idea addresses the issue of collapse in self-supervised learning, specifically when negative instances are not available. To prevent collapse, the proposal suggests using the eigenvalues of the output embedding space and ensuring they are all greater than 1.0. This aims to optimize the utilization of the embedding space. Additionally, the idea seeks to ensure that augmented variations of the same data point are mapped closer to each other.
and here are some paper abstracts that are relevant to this idea:

Paper ID 0: 
Recently, self-supervised learning methods have been proposed to learn a useful representation for visual detection of anomalous, unhealthy crops while a neural network classifies augmented images of normal instances, which are relatively easy-to-obtain. Their pip

In [85]:
print(parsed_response)

In the rapidly evolving field of self-supervised learning (SSL), the phenomenon known as 'collapse' presents a significant challenge, particularly in cases where negative instances are absent [Paper ID 1]. Recent research addresses this issue from multiple perspectives, among which one idea suggests managing eigenvalues of the output embedding space and ensuring they always exceed 1.0. 

Several studies focus on the strategy of contrasting normal and anomalous instances as a part of the SSL paradigm [Paper ID 0]. The use of augmented instances of normal samples as a strategy was proposed for visual detection problems, particularly for identifying unhealthy crops using neural networks. This is an interesting approach to self-supervised learning without negative examples but ensuring the augmentation variations of the same data are closer in embedding space.

Concerning the collapse issue during the absence of negative samples, the SimSiam method has gained attention due to its compellin

In [86]:
# refs = [''.join(['[', str(i), '], ', p]) for i, p in enumerate((papers_df['title']+ ', DOI: ' + papers_df['doi'] + ', (' + papers_df['year'].astype(str) + ')\n').values.tolist())]
# print(''.join(refs))
refs = [''.join(['[', str(i), '], ', p]) for i, p in enumerate((papers_df['title']+ ', (' + papers_df['year'].astype(str) + ')\n').values.tolist())]
print(''.join(refs))


[0], Exploiting Unlabeled Data to Improve Detection of Visual Anomalies in Soft Fruits, (2023)
[1], How Does SimSiam Avoid Collapse Without Negative Samples? A Unified Understanding with Self-supervised Contrastive Learning, (2022)
[2], Self-Supervised Predictive Learning: A Negative-Free Method for Sound Source Localization in Visual Scenes, (2022)
[3], Understanding self-supervised Learning Dynamics without Contrastive Pairs, (2021)
[4], Mean Shift for Self-Supervised Learning, (2021)
[5], TriNet: stabilizing self-supervised learning from complete or slow collapse, (2022)
[6], Event Camera Data Pre-training, (2023)
[7], Barlow Twins: Self-Supervised Learning via Redundancy Reduction, (2021)
[8], VICReg: Variance-Invariance-Covariance Regularization for Self-Supervised Learning, (2021)
[9], Joint Embedding Self-Supervised Learning in the Kernel Regime, (2022)
[10], What Should Be Equivariant In Self-Supervised Learning, (2022)
[11], Reduced order modeling with Barlow Twins self-superv

In [87]:
with open(working_dir + 'litrature_review_gpt4.txt', 'w') as f:
    f.write(parsed_response)
    f.write('\n\nReferences:\n')
    f.write(''.join(refs))
    

In [40]:
long_context_model.get_current_cost()+short_context_model.get_current_cost()

0.1974915

In [41]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# 1. Initialize BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# 2. Function to get embeddings
def get_embedding(text):
    tokens = tokenizer(text, return_tensors='pt', truncation=True, max_length=512, padding='max_length')
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs['pooler_output'].detach().numpy()

# 3. Compute the embedding for the idea
# idea_text = idea_text
idea_embedding = get_embedding(idea_text_summary)

# 4. Compute embeddings for abstracts and rank them
abstracts = combined_df['abstract'].dropna().values  # List of abstracts
abstract_embeddings = [get_embedding(abstract) for abstract in abstracts]

# 5. Calculate cosine similarities and rank
similarities = [cosine_similarity(idea_embedding, abstract_embedding)[0][0] for abstract_embedding in abstract_embeddings]
ranked_abstracts = sorted(zip(abstracts, similarities), key=lambda x: x[1], reverse=True)

# 6. Print ranked abstracts
for rank, (abstract, score) in enumerate(ranked_abstracts, 1):
    print(f"Rank: {rank}, Similarity Score: {score:.4f}")
    print(abstract)
    print("="*50)



Rank: 1, Similarity Score: 0.9609
Real world data is mostly unlabeled or only few instances are labeled. Manually labeling data is a very expensive and daunting task. This calls for unsupervised learning techniques that are powerful enough to achieve comparable results as semi-supervised/supervised techniques. Contrastive self-supervised learning has emerged as a powerful direction, in some cases outperforming supervised techniques. In this study, we propose, SelfGNN, a novel contrastive self-supervised graph neural network (GNN) without relying on explicit contrastive terms. We leverage Batch Normalization, which introduces implicit contrastive terms, without sacrificing performance. Furthermore, as data augmentation is key in contrastive learning, we introduce four feature augmentation (FA) techniques for graphs. Though graph topological augmentation (TA) is commonly used, our empirical findings show that FA perform as good as TA. Moreover, FA incurs no computational overhead, unlike

In [ ]:
# pip install -q transformers
from transformers import pipeline

checkpoint = "MBZUAI/LaMini-Flan-T5-77M"

model = pipeline('text2text-generation', model = checkpoint)
input_prompt = """Here is an idea: Lets say I have a neural network that maps a set of images from the same distribution. I want all of them to be mapped to the same embedding. To avoid collapse, I will use eigenvalues of the output embedding and ensure they are all larger 1.0, or maybe if they are multiplied to one another, the result is larger than 1.0.

How relevant this idea is to the following abstract of a paper:

Deep neural networks (DNNs), regardless of their impressive performance, are vulnerable to attacks from adversarial inputs and, more recently, Trojans to misguide or hijack the decision of the model. We expose the existence of an intriguing class of spatially bounded, physically realizable, adversarial examples— Universal NaTuralistic adversarial paTches—we call TnTs, by exploring the super set of the spatially bounded adversarial example space and the natural input space within generative adversarial networks. Now, an adversary can arm themselves with a patch that is naturalistic, less malicious-looking, physically realizable, highly effective—achieving high attack success rates, and universal. A TnT is universal because any input image captured with a TnT in the scene will: i) misguide a network (untargeted attack); or ii) force the network to make a malicious decision (targeted attack). Interestingly, now, an adversarial patch attacker has the potential to exert a greater level of control—the ability to choose a location independent, natural-looking patch as a trigger in contrast to being constrained to noisy perturbations—an ability is thus far shown to be only possible with Trojan attack methods needing to interfere with the model building processes to embed a backdoor at the risk discovery; but, still realize a patch deployable in the physical world. Through extensive experiments on the large-scale visual classification task, ImageNet with evaluations across its entire validation set of 50,000 images, we demonstrate the realistic threat from TnTs and the robustness of the attack. We show a generalization of the attack to create patches achieving higher attack success rates than existing state-of-the-art methods. Our results show the generalizability of the attack to different visual classification tasks (CIFAR-10, GTSRB, PubFig) and multiple state-of-the-art deep neural networks such as WideResnet50, Inception-V3 and VGG-16."""
input_prompt = 'Please let me know your thoughts on the given place and why you think it deserves to be visited: \n"Barcelona, Spain"'
generated_text = model(input_prompt, max_length=512, do_sample=True)[0]['generated_text']

print("Response", generated_text)
